# NBA Player Data

## 3 .csv files from:
https://www.kaggle.com/drgilermo/nba-players-stats

### Data collected by Omri Goldstein from Kaggle with 162 votes. Scraped from basketball-reference.com.

### Data Contents:
* player_data.csv: Column split csv with information about individual players. Columns: name, year_start, year_end, position, height, weight, birth_date, college. 8 header columns. Size: 4551 x 8

* Players.csv: Detailed information personal about player - height, weight, college, birthdate, birth_city, birth_state. Size: 3923 x 8

* Seasons_Stats.csv: Very detailed season stats for a given player. Describes important game statistics including: Team, games, games started, minutes played, player efficiency rating, true shooting percent, 3 point attempt rate, free throw rate, offensive rebound percentage, etc. Size - 24.7k x 53


# 1 dataset scraped by me from: 
https://stats.nba.com/draft/history/?Season={Season}

## Where {Season} is year in range 2007 - 2017


# Why this dataset?

## These datasets are required to answer the following questions


# Questions:
* How does draft order affect success? Look at top 3 draft picks for each year and analyze how they are currently playing.
* How do Lebron's stats look? Is Lebron slowing down with age or is he getting better? Are other players his age able to keep up?
* How many players have averaged superior numbers passed age 30?

# Scrape NBA draft data

In [74]:
import re
import requests
import pandas as pd
import html
import csv
from bs4 import BeautifulSoup 

In [94]:
season = 2015
url = f"https://nbahoopsonline.com/History/Leagues/NBA/drafts/{season}.html"

res = requests.get(url)
dom = BeautifulSoup(res.text)
dom.get_text()
# find <pre> tag where table is stored


selector = "pre"

# get contents of <pre> tag as string 
table = dom.find(selector).get_text()
table.strip()

print(table.split())



# table is array with long string
# split up array by round



['Rnd.', 'Pick', 'Player', 'Pos.', 'Team', 'School', '/', 'club', 'team', '1', '1', 'Karl-Anthony', 'Towns', 'C', 'Minnesota', 'Timberwolves', 'Kentucky', '(Fr.)', '1', '2', "D'Angelo", 'Russell', 'PG/SG', 'Los', 'Angeles', 'Lakers', 'Ohio', 'State', '(Fr.)', '1', '3', 'Jahlil', 'Okafor', 'C', 'Philadelphia', '76ers', 'Duke', '(Fr.)', '1', '4', 'Kristaps', 'Porziņģis', 'PF', 'New', 'York', 'Knicks', 'Baloncesto', 'Sevilla', '(Spain)', '1', '5', 'Mario', 'Hezonja', 'SG/SF', 'Orlando', 'Magic', 'FC', 'Barcelona', '(Spain)', '1', '6', 'Willie', 'Cauley-Stein', 'C', 'Sacramento', 'Kings', 'Kentucky', '(Jr.)', '1', '7', 'Emmanuel', 'Mudiay', 'PG', 'Denver', 'Nuggets', 'Guangdong', 'Southern', 'Tigers', '(China)', '1', '8', 'Stanley', 'Johnson', 'SF', 'Detroit', 'Pistons', 'Arizona', '(Fr.)', '1', '9', 'Frank', 'Kaminsky', 'C', 'Charlotte', 'Hornets', 'Wisconsin', '(Sr.)', '1', '10', 'Justise', 'Winslow', 'SF', 'Miami', 'Heat', 'Duke', '(Fr.)', '1', '11', 'Myles', 'Turner', 'C', 'Indiana', '

/home/kevin/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/kevin/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
